In [1]:
import numpy as np
import os
import tensorflow as tf
from keras.losses import SparseCategoricalCrossentropy
from tensorflow import keras
from keras import layers

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
#####################33
mfcc = True
####################33

zip_path = tf.keras.utils.get_file(
    origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
    fname='mini_speech_commands.zip',
    extract=True,
    cache_dir='.', cache_subdir='data')

data_dir = os.path.join('.', 'data', 'mini_speech_commands')
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
LABELS = ['stop', 'up', 'yes', 'right', 'left', 'no', 'down', 'go']
total = 8000
# eval_file = open('../kws_val_split(new).txt', 'r')
train_files = open('./kws_train_splitnew.txt', 'r')
test_files = open('./kws_test_splitnew.txt', 'r')

list_train = []
y_train = []
for i in train_files:
    list_train.append(i)
    tmp = i.replace('.\\data\\mini_speech_commands\\', '')
    loc_slash = tmp.find('\\')
    y_train.append(LABELS.index(tmp[:loc_slash]))
stripped_line = [s.rstrip() for s in list_train]
train_files = tf.convert_to_tensor(stripped_line)

test_list = []
y_test = []
for i in test_files:
    test_list.append(i)
    tmp = i.replace('.\\data\\mini_speech_commands\\', '')
    loc_slash = tmp.find('\\')
    y_test.append(LABELS.index(tmp[:loc_slash]))
test_files = tf.convert_to_tensor([s.rstrip() for s in test_list])
y_test = np.array(y_test)


class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
                 num_mel_bins=None, lower_frequency=None, upper_frequency=None,
                 num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                              frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                                       self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()

        return ds


def mlp_model(num_filter):
    mlp = keras.Sequential()
    mlp.add(layers.Flatten())
    mlp.add(layers.Dense(units=num_filter, activation='relu'))
    mlp.add(layers.Dense(units=num_filter, activation='relu'))
    mlp.add(layers.Dense(units=num_filter, activation='relu'))
    mlp.add(layers.Dense(units=8))
    mlp.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam')
    return mlp

alpha=0.75
def cnn_model(num_filter, momentom):
    cnn = keras.Sequential()
    cnn.add(layers.Conv2D(filters=num_filter, kernel_size=[3, 3], strides=strides, use_bias=False, activation='relu'))
    cnn.add(layers.BatchNormalization(momentum=momentom))
    cnn.add(layers.Conv2D(filters=num_filter, kernel_size=[3, 3], strides=[1, 1], use_bias=False, activation='relu'))
    cnn.add(layers.BatchNormalization(momentum=momentom))
    cnn.add(layers.Conv2D(filters=num_filter, kernel_size=[3, 3], strides=[1, 1], use_bias=False, activation='relu'))
    cnn.add(layers.BatchNormalization(momentum=momentom))
    cnn.add(layers.GlobalAveragePooling2D())
    cnn.add(layers.Dense(units=8))
    cnn.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
    return cnn


def ds_cnn_model():
    ds_cnn = keras.Sequential()
    ds_cnn.add(layers.Conv2D(filters=1024, kernel_size=[3, 3], strides=strides, use_bias=False, activation='relu'))
    ds_cnn.add(layers.BatchNormalization(momentum=0.1))
    ds_cnn.add(layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], use_bias=False))
    ds_cnn.add(layers.Conv2D(filters=1024, kernel_size=[1, 1], strides=[1, 1], use_bias=False, activation='relu'))
    ds_cnn.add(layers.BatchNormalization(momentum=0.1))
    ds_cnn.add(layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], use_bias=False))
    ds_cnn.add(layers.Conv2D(filters=1024, kernel_size=[1, 1], strides=[1, 1], use_bias=False, activation='relu'))
    ds_cnn.add(layers.BatchNormalization(momentum=0.1))
    ds_cnn.add(layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], use_bias=False))
    ds_cnn.add(layers.Conv2D(filters=1024, kernel_size=[1, 1], strides=[1, 1], use_bias=False, activation='relu'))
    ds_cnn.add(layers.BatchNormalization(momentum=0.1))
    ds_cnn.add(layers.GlobalAveragePooling2D())
    ds_cnn.add(layers.Dense(units=8))
    ds_cnn.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam')
    return ds_cnn


STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 650, 'frame_step': 350, 'mfcc': True,
                'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 30,
                'num_coefficients': 9}
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
test_ds = generator.make_dataset(test_files, False)
units = 8
model = cnn_model(num_filter=128,momentom=0.6)
model.fit(train_ds, verbose=1, epochs=30)
eval_result = np.argmax(model.predict(test_ds, verbose=2), axis=1)
eval_acc = sum(eval_result == y_test) / len(y_test)
print('Accuracy: ' +str(eval_acc))

Epoch 1/30
200/200 [==============================] - 20s 84ms/step - loss: 0.9465 - accuracy: 0.7061
Epoch 2/30
200/200 [==============================] - 1s 5ms/step - loss: 0.4043 - accuracy: 0.8875
Epoch 3/30
200/200 [==============================] - 1s 6ms/step - loss: 0.2756 - accuracy: 0.9219
Epoch 4/30
200/200 [==============================] - 1s 6ms/step - loss: 0.2044 - accuracy: 0.9422
Epoch 5/30
200/200 [==============================] - 1s 6ms/step - loss: 0.1489 - accuracy: 0.9622
Epoch 6/30
200/200 [==============================] - 1s 5ms/step - loss: 0.1080 - accuracy: 0.9747
Epoch 7/30
200/200 [==============================] - 1s 6ms/step - loss: 0.0822 - accuracy: 0.9822
Epoch 8/30
200/200 [==============================] - 1s 5ms/step - loss: 0.0650 - accuracy: 0.9870
Epoch 9/30
200/200 [==============================] - 1s 5ms/step - loss: 0.0480 - accuracy: 0.9912
Epoch 10/30
200/200 [==============================] - 1s 5ms/step - loss: 0.0491 - accuracy: 0.99

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 21, 7, 128)        1152      
_________________________________________________________________
batch_normalization (BatchNo (None, 21, 7, 128)        512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 5, 128)        147456    
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 5, 128)        512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 3, 128)        147456    
_________________________________________________________________
batch_normalization_2 (Batch (None, 17, 3, 128)        512       
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)               0

In [ ]:
#converting before pruning
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('./cnn_final.tflite', 'wb') as fp:
    fp.write(tflite_model)

Pruning

In [3]:
import tensorflow_model_optimization as tfmot
import tempfile
import pathlib
# prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
# model_for_pruning = prune_low_magnitude(cnn)

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 1
epochs = 15 #best values are either 9 or 10
validation_split = 0  # 10% of training set will be used for validation set.
#number of samples in the dataset is equal to 6400
num_images = 6400 * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.3,
                                                             final_sparsity=0.8,
                                                             begin_step=6400*5,
                                                             end_step=6400*15)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
                          loss=SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])

model_for_pruning.summary()
logdir = tempfile.mkdtemp()

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_ds,batch_size=batch_size,
                       epochs=epochs, callbacks=callbacks)
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
    test_ds, verbose=1)

print('Pruned test accuracy:', model_for_pruning_accuracy)
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')




tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)


C:\Users\aligh\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 21, 7, 128)        2306      
_________________________________________________________________
prune_low_magnitude_batch_no (None, 21, 7, 128)        513       
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 19, 5, 128)        294914    
_________________________________________________________________
prune_low_magnitude_batch_no (None, 19, 5, 128)        513       
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 17, 3, 128)        294914    
_________________________________________________________________
prune_low_magnitude_batch_no (None, 17, 3, 128)        513       
_________________________________________________________________
prune_low_magnitude_global_a (None, 128)               1

In [4]:
eval_result = np.argmax(model_for_pruning.predict(test_ds, verbose=1), axis=1)
eval_acc = sum(eval_result == y_test) / len(y_test)
print('accuracy: ' +str(eval_acc))

25/25 [==============================] - 0s 14ms/step
accuracy: 0.92375


In [5]:
import zlib, sys

# filename_in = "C:/Users/aligh/AppData/Local/Temp/tmp48515hyv.tflite"
filename_in='C:/Users/aligh/AppData/Local/Temp/tmpdy20i_i3.tflite'
filename_out = "./compressed_cnn_6_alpha_new.tflite.zlib"

with open(filename_in, mode="rb") as fin, open(filename_out, mode="wb") as fout:
    data = fin.read()
    compressed_data = zlib.compress(data,9)
    print(f"Original size: {sys.getsizeof(data)}")
    # Original size: 1000033
    print(f"Compressed size: {sys.getsizeof(compressed_data)}")
    # Compressed size: 1024

    fout.write(compressed_data)

Original size: 312769
Compressed size: 278101
